### Aufgaben die man mit Hilfe von Dictionaries löst

**Shift-Cipher**.  
Ein Text aus Kleinbuchstaben ohne Umlaute wird verschlüsselt, indem
jeder Buchstabe im Alphabet um die gleiche Distanz geschoben wird (nach z kommt wieder a).
Diese Distanz ist unser Code.
Zum Entschlüsseln wird jeder Buchstabe um die gleiche Distanz zurück geschoben.

**Shift-Cipher knacken**.  
Es gibt nur 26 verschiedene Codes.
Der Text wird mit jedem dieser Codes entschlüsselt.
Mit Hilfe eines Dictionaries bestimmen wir die relativen Häufigkeiten der Buchstaben.
Dann testen wir, bei welchem Code diese rel. Häufigkeiten am 
nächsten bei den rel. Häufigkeiten eines deutschsprachigen Textes liegen. 

Als Distanzmass verwenden wir die Wurzel der Summe der quadratischen Abweichungen der Häufigkeiten. Sind $\mathtt{d}$ und $\mathtt{e}$ solche Dicts, dann ist
$$
\text{distance}(d, e) := \sqrt{(\mathtt{d['A']-e['A']})^2+\ldots+\mathtt{d['Z']-e['Z']})^2}.
$$

Die relativen Häufigkeiten der Buchstaben eines deutschen bez. englischen Texts sind in den Dictionaries `letter_frequencies['de']` bez. `letter_frequencies['en']`
im Module `examples` gespeichert.

In [37]:
from examples import letter_frequency as lf


LFD = lf['de']  # letter frequency dict
''.join(list(LFD.keys())), sum(LFD.values())

('ABCDEFGHIJKLMNOPQRSTUVWXYZ', 0.9999999999999999)

In [38]:
def lf_distance(d_ref, d):
    diffs = [(d_ref[c] - d.get(c, 0)) ** 2 for c in d_ref]
    dist = (sum(diffs) / len(diffs)) ** 1/2
    return dist

In [43]:
from dict_tools import make_count_dict


def make_lfd(s):
    s = s.upper()
    cd = make_count_dict(s, cond=lambda c: 'A' <= c <= 'Z')
    tot = sum(cd.values())
    fd = {k: v/tot for k, v in cd.items()}
    kv_pairs = sorted(fd.items(), key=lambda x: x[1], reverse=True)
    fd = dict(kv_pairs)
    return fd

In [83]:
text = '''Ein Text aus Kleinbuchstaben ohne Umlaute wird verschlüsselt, \
indem jeder Buchstabe im Alphabet um die gleiche Distanz geschoben wird.'''


fd = make_lfd(text)
lf_distance(LFD, fd)

0.00010643581836984808

In [24]:
def make_cipher_dict(shift):
    abc = 'abcdefghijklmnopqrstuvwxyz'
    abc_shifted = abc[shift:] + abc[:shift]
    return dict(zip(abc, abc_shifted))


def encript(plain_text, code):
    d = make_cipher_dict(code)
    return ''.join(d.get(c, c) for c in plain_text)


def lowercase_without_umlauts(s):
    subs = {'ä': 'ae',  'ö': 'oe', 'ü': 'ue'}
    s = s.lower()
    for old, new in subs.items():
        s = s.replace(old, new)
    return s

In [25]:
s = 'hello'
c = encript(s, 7)
c

'olssv'

In [26]:
encript(c, -7)

'hello'

In [75]:
text = '''Ein Text aus Kleinbuchstaben ohne Umlaute wird verschlüsselt, \
indem jeder Buchstabe im Alphabet um die gleiche Distanz geschoben wird.'''

s = lowercase_without_umlauts(text)
cipher_text = encript(s, 17)

In [68]:
encript(cipher_text, -17)

'ein text aus kleinbuchstaben ohne umlaute wird verschluesselt, indem jeder buchstabe im alphabet um die gleiche distanz geschoben wird.'

In [97]:
distance_code_pairs = []
for code in range(26):
    plain_text = encript(cipher_text, -code)
    fd = make_lfd(plain_text)
    dist = lf_distance(LFD, fd)
    distance_code_pairs.append((dist, code))

shift = min(distance_code_pairs)[1]
encript(cipher_text, -shift)

'ein text aus kleinbuchstaben ohne umlaute wird verschluesselt, indem jeder buchstabe im alphabet um die gleiche distanz geschoben wird.'

In [98]:
def crack_code(cipher_text):
    distance_code_pairs = []
    for shift in range(26):
        plain_text = encript(cipher_text, -shift)
        fd = make_lfd(plain_text)
        dist = lf_distance(LFD, fd)
        distance_code_pairs.append((dist, shift))

    shift = min(distance_code_pairs)[1]
    msg = encript(cipher_text, -shift)
    return msg

In [99]:
crack_code(cipher_text)

'ein text aus kleinbuchstaben ohne umlaute wird verschluesselt, indem jeder buchstabe im alphabet um die gleiche distanz geschoben wird.'

In [100]:
s = lowercase_without_umlauts(text)
[i for i in range(26) if s != crack_code(encript(s, i))]


[]